## 数据集处理

## 1. 获取沪深300股票代码

In [1]:
import requests

#爬取沪深300股票代码
def spider_for_hs300():
    page=1
    hs300=[]
    while(page<=6):
        url = 'https://datainterface.eastmoney.com/EM_DataCenter/js.aspx?'
        key_dict = {
            'cb': 'jQuery112305522721612895851_1626420027301',
            'st': '1',
            'sr': '-1',
            'ps': '50',
            'p': page,
            'type': 'SHSZZS',
            'sty': 'SHSZZS',
            'code': '000300'
        }
        data = requests.get(url, params=key_dict)
        res=data.text
        for i in range(len(res)):
            if((i>0)&(res[i-1]=='"')&(res[i]>='0')&(res[i]<='9')):
                cnt=0
                ans=""
                while(cnt<6):
                    ans=ans+res[i]
                    cnt+=1
                    i+=1
                hs300.append(ans)
        page+=1
    return hs300

res = spider_for_hs300()
print(res)
print(len(set(res)))

['688396', '688363', '688169', '688126', '688111', '688036', '688012', '688009', '688008', '603993', '603986', '603939', '603899', '603882', '603833', '603806', '603799', '603659', '603658', '603517', '603501', '603392', '603369', '603338', '603288', '603259', '603233', '603195', '603160', '603087', '603019', '601998', '601995', '601990', '601989', '601988', '601985', '601939', '601933', '601919', '601916', '601901', '601899', '601888', '601881', '601878', '601877', '601872', '601857', '601838', '601818', '601816', '601808', '601800', '601799', '601788', '601766', '601727', '601698', '601696', '601688', '601669', '601668', '601658', '601633', '601628', '601618', '601607', '601601', '601600', '601555', '601398', '601390', '601377', '601360', '601336', '601328', '601319', '601318', '601288', '601238', '601236', '601231', '601229', '601225', '601216', '601211', '601186', '601169', '601166', '601162', '601155', '601138', '601111', '601108', '601100', '601088', '601077', '601066', '601021',

## 2. 筛出沪深300成分的股票数据

In [2]:
import pandas as pd

#从所有数据中筛选沪深300股票数据
data = pd.read_csv("./data/data.csv")
ans = pd.DataFrame(columns=('kdcode','dt','volume','turnover','adj_open','adj_close','adj_high','adj_low','adj_downprice','adj_upprice','pct_chg'))

for i in range(300):
    da = data.loc[data['kdcode']==res[i]+".SZ"]
    if da.empty:
        da = data.loc[data['kdcode']==res[i]+".SH"]
    ans = pd.concat([da, ans])
    
ans.tail()

,kdcode,dt,volume,turnover,adj_open,adj_close,adj_high,adj_low,adj_downprice,adj_upprice,pct_chg
794220,603833.SH,2020-12-25,3067151.0,365870249.0,176.289910,171.287863,177.278853,168.607683,158.144948,193.288270,-2.5204
794221,603833.SH,2020-12-28,2964080.0,369112149.0,170.671565,180.231351,181.994250,170.556905,154.160510,188.415216,5.2213
794222,603833.SH,2020-12-29,1376880.0,173827156.0,178.726437,179.686715,182.725208,178.181802,162.215382,198.261652,-0.3022
794223,603833.SH,2020-12-30,1169645.0,149186731.0,180.947977,182.653546,184.717428,179.686715,161.713744,197.659687,1.6511
794224,603833.SH,2020-12-31,1769739.0,234753412.0,184.602767,192.772300,193.761243,182.768206,164.393924,200.913167,5.5399


* 没有重复数据 无需去重

In [3]:
# 去重
print(len(ans))
ans.drop_duplicates(subset=['kdcode', 'dt'],keep='first',inplace=True)
print(len(ans))

# ans.to_csv("./data/hs300.csv", index=False)

94466
94466


## 3. 分割训练集、测试集

In [4]:
import re

datalist = ans.values.tolist()
trainSet = []
testSet = []

for data_i in datalist:
    if re.match('2020', data_i[1]) == None:
        trainSet.append(data_i)
    else:
        testSet.append(data_i)
        
print("训练集共 " + str(len(trainSet)) + " 条")
print("测试集共 " + str(len(testSet)) + " 条")

testData = pd.DataFrame(testSet, columns=('kdcode','dt','volume','turnover','adj_open','adj_close','adj_high','adj_low','adj_downprice','adj_upprice','pct_chg'))
# testData.to_csv("./data/test.csv", index=False)

训练集共 74561 条
测试集共 19905 条


## 4. 填补训练集停牌数据
按照**日期**填补训练集中除了周六日外缺少的数据

In [5]:
import pandas as pd
from datetime import datetime
from datetime import timedelta


# 判断闰年
def isRunYear(year):
    if year % 100 == 0:
        if year % 400 == 0:
            return 1
    else:
        if year % 4 == 0:
            return 1
    return 0


# 判断是一年中的第几天
def dayNum(year, month, day):
    ret = 0
    ping = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    run = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    if isRunYear(year):
        for i in range(month - 1):
            ret = ret + run[i]
    else:
        for i in range(month - 1):
            ret = ret + ping[i]
    return ret + day


# 判断是否是周末
def isSunOrSat(year, month, day):
    S = (year + (year - 1) // 4 - (year - 1) // 100 + (year - 1) // 400) % 7  # 第year年第一天是周S
    weekday = (dayNum(year, month, day) + S - 1) % 7
    if weekday == 0 or weekday == 6:
        return 1
    else:
        return 0


# trainSet = pd.read_csv("train.csv").values.tolist()
datetime_tmp = pd.to_datetime('2022-01-01')
date_tmp = datetime.date(datetime_tmp)
data_tmp = []
trainData = []

for data_i in trainSet:
    now_date = datetime.date(pd.to_datetime(data_i[1]))
    if now_date < date_tmp:
        trainData.append(data_i)
        date_tmp = now_date
        data_tmp = data_i.copy()
    else:
        daydelta = now_date - date_tmp
        if daydelta.days != 1:
            for i in range(1, daydelta.days):
                add_day = date_tmp + timedelta(days=i)
                if isSunOrSat(add_day.year, add_day.month, add_day.day) != 1:
                    add_data = data_tmp.copy()
                    add_data[1] = str(add_day)
                    trainData.append(add_data)
        date_tmp = now_date
        data_tmp = data_i.copy()
        trainData.append(data_i)

print("填充后训练集数据共 " + str(len(trainData)) + " 条")
trainData = pd.DataFrame(trainData, columns=('kdcode', 'dt', 'volume', 'turnover', 'adj_open', 'adj_close', 'adj_high', 'adj_low', 'adj_downprice', 'adj_upprice','pct_chg'))
# trainData.to_csv("./data/train.csv", index=False)

填充后训练集数据共 81574 条


## 5. 数据归一化

In [6]:
print(trainData.shape)
print(testData.shape)
trainData_df1 = trainData.iloc[:, :2]
trainData_df2 = trainData.iloc[:, 2:]
testData_df1 = testData.iloc[:, :2]
testData_df2 = testData.iloc[:, 2:]

print(trainData_df1.shape)
print(trainData_df2.shape)
print(testData_df1.shape)
print(testData_df2.shape)

testData_df1.tail()
# testData_df2.tail()

(81574, 11)
(19905, 11)
(81574, 2)
(81574, 9)
(19905, 2)
(19905, 9)


,kdcode,dt
19900,603833.SH,2020-12-25
19901,603833.SH,2020-12-28
19902,603833.SH,2020-12-29
19903,603833.SH,2020-12-30
19904,603833.SH,2020-12-31


In [7]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
# 归一化训练集
ProcessedTrainData = min_max_scaler.fit_transform(trainData_df2)
ProcessedTrainDataDf2 = pd.DataFrame(ProcessedTrainData, columns=trainData_df2.columns)
print(ProcessedTrainDataDf2.shape)

trainData_df = pd.concat([trainData_df1,ProcessedTrainDataDf2],axis=1)
print(trainData_df.shape)
trainData_df.tail()

(81574, 9)
(81574, 11)


,kdcode,dt,volume,turnover,adj_open,adj_close,adj_high,adj_low,adj_downprice,adj_upprice,pct_chg
81569,603833.SH,2019-12-25,0.000656,0.003380,0.011902,0.011900,0.012004,0.011738,0.012089,0.012089,0.195567
81570,603833.SH,2019-12-26,0.001215,0.006451,0.011918,0.012290,0.012290,0.011851,0.012149,0.012150,0.245575
81571,603833.SH,2019-12-27,0.001029,0.005549,0.012323,0.012289,0.012496,0.012191,0.012541,0.012540,0.186047
81572,603833.SH,2019-12-30,0.001204,0.006459,0.012320,0.012429,0.012453,0.012237,0.012540,0.012539,0.206831
81573,603833.SH,2019-12-31,0.000762,0.004127,0.012570,0.012550,0.012561,0.012419,0.012679,0.012679,0.203736


In [8]:
# 归一化测试集
ProcessedTestData = min_max_scaler.fit_transform(testData_df2)
ProcessedTestDataDf2 = pd.DataFrame(ProcessedTestData, columns=testData_df2.columns)
print(ProcessedTestDataDf2.shape)

testData_df = pd.concat([testData_df1,ProcessedTestDataDf2],axis=1)
print(testData_df.shape)
testData_df.tail()

(19905, 9)
(19905, 11)


,kdcode,dt,volume,turnover,adj_open,adj_close,adj_high,adj_low,adj_downprice,adj_upprice,pct_chg
19900,603833.SH,2020-12-25,0.002007,0.013505,0.011714,0.011052,0.011437,0.011206,0.011724,0.011725,0.250670
19901,603833.SH,2020-12-28,0.001925,0.013635,0.011333,0.011640,0.011747,0.011338,0.011423,0.011424,0.508227
19902,603833.SH,2020-12-29,0.000650,0.005786,0.011879,0.011604,0.011795,0.011855,0.012032,0.012032,0.324467
19903,603833.SH,2020-12-30,0.000483,0.004795,0.012029,0.011799,0.011926,0.011957,0.011994,0.011995,0.389451
19904,603833.SH,2020-12-31,0.000965,0.008235,0.012276,0.012465,0.012520,0.012166,0.012196,0.012196,0.518827


In [9]:
# 把收盘价放到最后以便于后续操作
order = ['kdcode', 'dt', 'volume', 'turnover', 'adj_open', 'adj_high', 'adj_low', 'adj_downprice', 'adj_upprice','pct_chg', 'adj_close']
trainData_df = trainData_df[order]
testData_df = testData_df[order]

trainData_df.tail()
# testData_df.tail()

,kdcode,dt,volume,turnover,adj_open,adj_high,adj_low,adj_downprice,adj_upprice,pct_chg,adj_close
81569,603833.SH,2019-12-25,0.000656,0.003380,0.011902,0.012004,0.011738,0.012089,0.012089,0.195567,0.011900
81570,603833.SH,2019-12-26,0.001215,0.006451,0.011918,0.012290,0.011851,0.012149,0.012150,0.245575,0.012290
81571,603833.SH,2019-12-27,0.001029,0.005549,0.012323,0.012496,0.012191,0.012541,0.012540,0.186047,0.012289
81572,603833.SH,2019-12-30,0.001204,0.006459,0.012320,0.012453,0.012237,0.012540,0.012539,0.206831,0.012429
81573,603833.SH,2019-12-31,0.000762,0.004127,0.012570,0.012561,0.012419,0.012679,0.012679,0.203736,0.012550


In [10]:
# 保存训练集、测试集数据
trainData_df.to_csv("./data/train.csv", index=False)
testData_df.to_csv("./data/test.csv", index=False)